<a href="https://colab.research.google.com/github/MikhailBolt/school_DS/blob/main/%D0%A8%D0%BA%D0%BE%D0%BB%D0%B0_DS_%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%5B%D0%91%D0%BE%D0%BB%D1%82%D1%8B%D1%88%D0%B5%D0%B2_%D0%9C%D0%B8%D1%85%D0%B0%D0%B8%D0%BB%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Дейтинг-индустрия
https://cloud.mail.ru/public/yvsP/gcRYW5qC4


При взаимном лайке возникает возможность общения

Для упрощения можно допустить: если rating >=6, то это лайк, иначе - дизлайк

Необходимо написать код, который поможет найти для следуюших пользователей, которые они с максимальной вероятностью залайкают (за вычетом известных в датасете пар). Для каждого id - не более 10 пользователей

id: 98447, 111944, 111930, 82091, 125299, 125298, 17335, 60356


2. Дополнительно:
a. Необходимо сформировать 2 колоды пользователей для каждого:
i. Те, которых пользователь с максимальной вероятностью лайкнет
ii. Те, с кем максимальной вероятностью будет мэтч (взаимный
лайк)
b. Упаковать в сервис по API (flask)

In [72]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [73]:
# Заргужаем даныне
# В датасете данные о пользователе
# 1ая колонка id
# 2ая колонка gender: "M" - male, "F" - female,  "U" - unknown
# по остальным колонкам недоступна информация
#df_info = pd.read_csv('info.csv')
# В датасете данные, как пользователь с id_from оценил (rating) пользователя id_to
#df_ratings = pd.read_csv("rec-libimseti-dir.edges", delim_whitespace=True, skiprows=1, names=["id_from", "id_to", "rating"])
# Загрузка данных
# В датасете данные, как пользователь с id_from оценил (rating) пользователя id_to
users  = pd.read_csv('info.csv')
ratings  = pd.read_csv("rec-libimseti-dir.edges", delim_whitespace=True, skiprows=1, names=["id_from", "id_to", "rating"])

<ipython-input-73-b35acdf8cc97>:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ratings  = pd.read_csv("rec-libimseti-dir.edges", delim_whitespace=True, skiprows=1, names=["id_from", "id_to", "rating"])


In [74]:
users.columns = users.columns.str.strip()

In [77]:
# Посмотрим на первые несколько строк каждого файла для анализа структуры данных
users.head(), users.columns

(   id gender  1   2   3   4   5   6   7   8   9  10  11        12
 0   1      F  1  45  90  15   8  13  63  57  96  42  87  0.934487
 1   2      F  1  46  81   6   9  31  22  79  82  75  53  0.675761
 2   3      U  4  69  64   2  18  44  54  80  50  89  38  0.927339
 3   4      F  3  46  90   9  42  54  59  86  60  65  25  0.176650
 4   5      F  3  79  83   1  45  57   1  58  34  23  33  0.553003,
 Index(['id', 'gender', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
        '12'],
       dtype='object'))

In [78]:
ratings.head()

,id_from,id_to,rating
0,1,133,8
1,1,720,6
2,1,971,10
3,1,1095,7
4,1,1616,10


In [79]:
def get_filtered_ids(id_target):
    # Фильтрация строк, где id_from соответствует id_target и rating >= 6
    filtered_data = ratings[(ratings['id_from'] == id_target) & (ratings['rating'] >= 6)]

    # Извлечение значений из столбца id_to
    ids_to = filtered_data['id_to'].tolist()
    return ids_to

In [80]:
# Пример использования функции с ID_target = 1
ID_target = 1
result = get_filtered_ids(ID_target)

In [81]:
print(f"Список id_to для ID_target = {ID_target} с rating >= 6:", result)

Список id_to для ID_target = 1 с rating >= 6: [133, 720, 971, 1095, 1616, 1978, 2145, 2211, 3751, 4633, 6518, 7576, 7724, 8305, 8923, 9345, 9729, 10148, 10811, 11747, 13210, 14604, 14985, 15123, 15530, 16480, 18032, 18287, 18345, 18444, 18562, 19006, 19727, 20253, 20737, 21184, 21256, 21642, 22319, 22853, 22915, 24294, 24683, 25731, 26084, 26236, 32136, 32953, 33171, 33205, 33960, 34358, 34406, 35647, 37184, 37773, 38181, 38356, 38444, 38868, 39095, 39414, 39681, 40377, 41925, 43407, 43836, 44077, 44510, 47314, 48470, 48734, 49038, 49129, 49466, 50408, 50688, 51399, 51524, 51913, 53456, 53933, 54380, 54929, 56397, 56792, 57443, 57839, 58367, 58839, 59161, 60741, 61157, 62095, 63063, 64257, 64272, 65602, 69882, 70694, 72324, 73281, 73318, 73346, 73611, 75050, 75243, 76195, 78232, 79237, 80005, 80463, 80672, 81703, 82828, 83622, 83855, 84403, 84587, 87068, 88118, 88179, 88886, 88892, 91098, 91102, 91208, 93105, 93385, 93887, 94117, 94797, 94833, 94997, 97523, 97655, 98556, 98715, 99285, 

In [82]:
filtered_ids_to = result

In [83]:
# Копирование данных в новую таблицу
new_info = users.copy()

In [85]:
# Фильтрация данных из info.csv по значениям в списке filtered_ids_to
filtered_info = new_info[new_info['id'].isin(filtered_ids_to)]

In [86]:
# Вывод результата
print(filtered_info)

            id gender  1   2   3   4   5   6   7   8   9  10  11        12
132        133      M  3  51  95  21   0  19  49  13  47  61  46  0.217595
719        720      F  1  77  15   1  53  51  25  38  18   7  31  0.009226
970        971      M  3  34  30   2  42  27  11  23  97  48  65  0.057394
1094      1095      M  2  37  22  10  36   2  26  32  13  81  67  0.322294
1615      1616      M  3  79  24  19  42  58  68  74  26  22  78  0.927649
...        ...    ... ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..       ...
212502  212503      U  4  61  23   6  18  37  47  52  73  47  84  0.975736
213125  213126      M  3  39  14  19   4   7  60  68  32   6  62  0.943891
213357  213358      U  2  18  52  13  53  49  53   9  21  12  97  0.290535
213950  213951      M  5  74  54  13   9   5  42  49  92  32  66  0.901535
217430  217431      M  5  56  94  16  50  59   1  20  52  40  15  0.461746

[228 rows x 14 columns]


In [91]:
# Убираем идентификатор 'ID' из расчета корреляции
# Предполагаем, что остальные столбцы - это характеристики, по которым будет рассчитана корреляция
filtered_info_features = filtered_info.drop(columns=['id','gender'])
info_features = new_info.drop(columns=['id','gender'])


In [92]:
info_features

,1,2,3,4,5,6,7,8,9,10,11,12
0,1,45,90,15,8,13,63,57,96,42,87,0.934487
1,1,46,81,6,9,31,22,79,82,75,53,0.675761
2,4,69,64,2,18,44,54,80,50,89,38,0.927339
3,3,46,90,9,42,54,59,86,60,65,25,0.176650
4,3,79,83,1,45,57,1,58,34,23,33,0.553003
...,...,...,...,...,...,...,...,...,...,...,...,...
220965,5,26,65,5,7,18,18,31,65,55,50,0.201921
220966,1,85,60,16,30,53,60,53,15,37,58,0.624036
220967,1,70,73,22,33,57,88,23,2,22,86,0.867339
220968,2,40,45,21,28,27,96,75,77,60,48,0.144322


In [93]:
filtered_info_features

,1,2,3,4,5,6,7,8,9,10,11,12
132,3,51,95,21,0,19,49,13,47,61,46,0.217595
719,1,77,15,1,53,51,25,38,18,7,31,0.009226
970,3,34,30,2,42,27,11,23,97,48,65,0.057394
1094,2,37,22,10,36,2,26,32,13,81,67,0.322294
1615,3,79,24,19,42,58,68,74,26,22,78,0.927649
...,...,...,...,...,...,...,...,...,...,...,...,...
212502,4,61,23,6,18,37,47,52,73,47,84,0.975736
213125,3,39,14,19,4,7,60,68,32,6,62,0.943891
213357,2,18,52,13,53,49,53,9,21,12,97,0.290535
213950,5,74,54,13,9,5,42,49,92,32,66,0.901535


In [102]:
mean_values = filtered_info_features.mean().to_frame().T
mean_values

,1,2,3,4,5,6,7,8,9,10,11,12
0,2.95614,53.263158,47.539474,11.464912,28.614035,29.815789,49.399123,52.416667,51.350877,53.846491,52.114035,0.496134


In [103]:
# Вычисляем корреляцию с каждой строкой
correlations = []
for i, row in info_features.iterrows():
    # Вычисляем корреляцию строки mean_values со строкой row
    corr = row.corr(mean_values.iloc[0])
    correlations.append((i, corr))


In [104]:
# Сортируем корреляции и выбираем топ-10
top_10_correlations = sorted(correlations, key=lambda x: x[1], reverse=True)[:10]

In [108]:
# Выводим id пользователя и значения корреляции
print("Топ-10 строк с наибольшей корреляцией:")
for index, corr in top_10_correlations:

    print(f"id: {index+1}, Корреляция: {corr}")

Топ-10 строк с наибольшей корреляцией:
id: 55977, Корреляция: 0.9875836330934695
id: 135080, Корреляция: 0.9845229538958966
id: 32562, Корреляция: 0.984232794003118
id: 115065, Корреляция: 0.9840433919202188
id: 94298, Корреляция: 0.9826683088919749
id: 91483, Корреляция: 0.9819184787836739
id: 97589, Корреляция: 0.9816644457448787
id: 152748, Корреляция: 0.9813352924733263
id: 44803, Корреляция: 0.9805834474816583
id: 37617, Корреляция: 0.9804611644876813
